In [1]:
import torch
import torchvision
import numpy as np
import pandas as pd
import seaborn as sn
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torchvision.transforms as transforms

from torch.autograd import Variable
from sklearn.metrics import confusion_matrix

In [6]:
#Data Loading
train = pd.read_csv('../mnist/mnist_train.csv')
test = pd.read_csv('../mnist/mnist_test.csv')

In [9]:
#Validation Set

#Validation contains 100 Samples
df_train = np.array(train)
df_validation = df_train[:100, :]

#Remove Samples from train set
df_train = df_train[100:, :] 

In [10]:
#Data Processincg
test_fraction = 10
n_samples = int(df_train.shape[0]/sample_fraction)
df_train = df_train[:n_samples, :]

# Create x_train, y_train with adequate sizes
x_train = df_train[:, 1:] / 255
x_train = np.reshape(x_train, (n_samples ,28, 28))
y_train = df_train[:n_samples, 0]

df_test = np.array(test)
df_test = df_test[:int(n_samples), :]

x_test = df_test[:, 1:] / 255
x_test = np.reshape(x_test, (x_test.shape[0], 28, 28))
y_test = df_test[:, 0]

#Convert sets to tensor
x_train = torch.FloatTensor(x_train)
y_train = torch.from_numpy(y_train)

x_test = torch.FloatTensor(x_test)
y_test = torch.from_numpy(y_test)

print('Number of train samples: {}'.format(n_samples))

Number of train samples: 5989


In [11]:
batch_size = 2
train_set = torch.utils.data.TensorDataset(x_train, y_train)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)

validation_set = torch.utils.data.TensorDataset(x_validation, y_validation)
validation_loader = torch.utils.data.DataLoader(validation_set, batch_size=batch_size, shuffle=False)

In [14]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        #Convolution Layers
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d(p=0.25)
        
        #Linear layers
        self.fc1 = nn.Linear(320, 120)
        self.fc1_drop = nn.Dropout(p=0.25)
        self.fc2 = nn.Linear(120, 10)
    
    def foward(self, x):
        x = F.relu(F.max_pool2d(input=self.conv1(x), kernel_size=2))
        x = F.relu(F.max_pool2d(input=self.conv2_drop(self.conv2(x)), kernel_size=2))
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1_drop(self.fc1(x)))
        x = F.relu(self.fc2(x))
        return F.log_softmax(x)
                   
    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        num_features = size.reduce(lambda x: num_features * x)
        return num_features
net = Net()
net     

Net(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2_drop): Dropout2d(p=0.25)
  (fc1): Linear(in_features=320, out_features=120, bias=True)
  (fc1_drop): Dropout(p=0.25)
  (fc2): Linear(in_features=120, out_features=10, bias=True)
)